In [42]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer # Returns word to its origin
import numpy as np # For scientific computing and data analysis
import tensorflow as tf
from tensorflow import keras
import json
from tensorflow.keras.callbacks import TensorBoard

In [19]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ahmed_2tllwxa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ahmed_2tllwxa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
# load the intents dataset
with open('data.json', "rb") as file:
  #/content/drive/MyDrive/Colab Notebooks/ChatbotDataset.json
    intents = json.load(file)

In [21]:
# create lemmatizer object
lemmatizer = WordNetLemmatizer()
# WordNetLemmatizer is a tool used in natural language processing to convert words to their base or root form

In [22]:
# Difference between PorterStemmer and WordNetLemmatizer:
# PorterStemmer is a rule-based algorithm that removes the suffixes from words to generate the root form.
# WordNetLemmatizer uses a dictionary-based approach and takes into account the context and part of speech 
# of the word to generate the base form.

In [23]:
# create empty lists to hold the tokenized words, classes, and documents
words = []
classes = []
documents = []
ignore_words = ['?', '!']

In [24]:
# loop through each sentence in the intents patterns and tokenize the words
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add the tokenized words to the words list
        words.extend(w)
        # add the tokenized words and class to the documents list
        documents.append((w, intent['tag']))
        # add the class to the classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [25]:
# lemmatize the words and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))


In [26]:
# sort the classes
classes = sorted(list(set(classes)))

In [36]:
# create the training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    # initialize the bag of words for each document
    bag = []
    # tokenize each word in the document and lemmatize it
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    # create the bag of words for this document
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # create the output row
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # add the training data to the final list
    training.append([bag, output_row])
    

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [37]:
# create the training data
train_x = []
train_y = []

for doc in documents:
    # initialize the bag of words for each document
    bag = []
    # tokenize each word in the document and lemmatize it
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in doc[0]]
    # create the bag of words for this document
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # create the output row
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    # add the bag-of-words representation and output row to their respective lists
    train_x.append(bag)
    train_y.append(output_row)

# convert the lists to numpy arrays
train_x = np.array(train_x)
train_y = np.array(train_y)

# shuffle the training data
indices = np.arange(train_x.shape[0])
np.random.shuffle(indices)
train_x = train_x[indices]
train_y = train_y[indices]

# Rest of the code...


In [38]:
# len(training)
print(training)

[]


In [29]:
# shuffle the training data and convert it to a numpy array
np.random.shuffle(training)
training = np.array(training)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (109, 2) + inhomogeneous part.

In [14]:
# split the training data into input and output sets
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [40]:
# define the model architecture
model = keras.Sequential([
    keras.layers.Dense(128, input_shape=(len(train_x[0]),), activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(train_y[0]), activation='softmax')
])

In [41]:
# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [43]:
log_dir = "../Run/FeedForward"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [51]:
# train the model for 100 epochs
# model.fit(np.array(train_x), np.array(train_y), epochs=140, batch_size=5, callbacks=[tensorboard_callback])
history = model.fit(np.array(train_x), np.array(train_y),validation_data=([np.array(train_x) , np.array(train_y)]), batch_size=64, epochs=100, callbacks=[tensorboard_callback]) 

Epoch 1/100
2/2 [==============================] - 1s 663ms/step - loss: 0.0348 - accuracy: 0.9817 - val_loss: 0.0256 - val_accuracy: 0.9817
Epoch 2/100
2/2 [==============================] - 0s 389ms/step - loss: 0.0232 - accuracy: 1.0000 - val_loss: 0.0256 - val_accuracy: 0.9817
Epoch 3/100
2/2 [==============================] - 0s 389ms/step - loss: 0.0305 - accuracy: 0.9908 - val_loss: 0.0256 - val_accuracy: 0.9817
Epoch 4/100
2/2 [==============================] - 0s 353ms/step - loss: 0.0290 - accuracy: 0.9908 - val_loss: 0.0256 - val_accuracy: 0.9817
Epoch 5/100
2/2 [==============================] - 0s 290ms/step - loss: 0.0359 - accuracy: 0.9817 - val_loss: 0.0256 - val_accuracy: 0.9817
Epoch 6/100
2/2 [==============================] - 0s 317ms/step - loss: 0.0828 - accuracy: 0.9633 - val_loss: 0.0256 - val_accuracy: 0.9817
Epoch 7/100
2/2 [==============================] - 0s 334ms/step - loss: 0.0290 - accuracy: 0.9817 - val_loss: 0.0256 - val_accuracy: 0.9817
Epoch 8/100
2

In [18]:
4# Define function to predict intent
def predict_intent(sentence):
    sentence_words = nltk.word_tokenize(sentence.lower())
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    bag = [0] * len(words)
    for word in sentence_words:
        for i, w in enumerate(words):
            if w == word:
                bag[i] = 1
    res = model.predict(np.array([bag]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for result in results:
        return_list.append((classes[result[0]], result[1]))
    return return_list

In [19]:
# Test model
print(predict_intent("Hi, how are you?"))
print(predict_intent("What's on the menu today?"))
print(predict_intent("Thanks"))
print(predict_intent("What are some popular places to visit in Egypt?"))

1/1 [==============================] - 0s 95ms/step
[('greeting', 0.9999962)]
1/1 [==============================] - 0s 26ms/step
[('weather', 0.99999976)]
1/1 [==============================] - 0s 29ms/step
[('thankyou', 1.0)]
1/1 [==============================] - 0s 26ms/step
[('places_to_visit', 1.0)]


In [20]:
# save the model
# model.save('/content/drive/MyDrive/Colab Notebooks/chatbot_model.h5')
model.save('/content/chatbot_model.h5')

In [21]:
# load the model
# model = keras.models.load_model('/content/drive/MyDrive/Colab Notebooks/chatbot_model.h5')
model = keras.models.load_model('/content/chatbot_model.h5')

In [22]:
# define a function to process user input and get the chatbot response
def chatbot_response(text):
    # tokenize the user input
    input_data = nltk.word_tokenize(text)
    # lemmatize the input words and remove duplicates
    input_data = [lemmatizer.lemmatize(word.lower()) for word in input_data]
    # create a bag of words for the input data
    bag_of_words = [0] * len(words)
    for word in input_data:
        for i, w in enumerate(words):
            if w == word:
                bag_of_words[i] = 1
    # predict the class of the input data using the trained model
    result = model.predict(np.array([bag_of_words]))[0]
    # get the index of the predicted class
    index = np.argmax(result)
    # get the tag of the predicted class
    tag = classes[index]

    # if the predicted class has a confidence score lower than 0.5, return a default response
    if result[index] < 0.5:
        return "I'm sorry, I don't understand. Can you please try again with different words?"

    # loop through the intents and find the one with the matching tag
    for intent in intents['intents']:
        if intent['tag'] == tag:
            # choose a random response from the list of responses
            response = np.random.choice(intent['responses'])
            return response

In [24]:
print("Press 0 if you don't want to chat with our ChatBot.")
# test the chatbot model
while True:
    # get user input
    user_input = input("''You: ")
    # Condition if the user press 0 it terminates
    if user_input == "0":
     break
    # Predict the intent of input
    print(predict_intent(user_input))
    # get chatbot response
    chatbot_output = chatbot_response(user_input)
    print("Chatbot:", chatbot_output)

Press 0 if you don't want to chat with our ChatBot.
''You: 0
